# Policy Impact Analysis
This notebook iterates through 6 different policy reforms and calculates their 10-year budgetary impacts.

In [9]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

## Define Policy Reform Functions
Each function returns a Reform object for different policy options

In [10]:
# Option 1
def get_repeal_social_security_reform():
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        }
        }, country_id="us")

# Policy 2
def get_flat_ss_tax_reform():
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2024-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 0
        }
        }, country_id="us")

# Policy 3
def get_flat_ss_tax_with_senior_deduction_extension_reform():
    # Placeholder
    # Replace with actual PolicyEngine reform parameters
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2024-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2024-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.senior_deduction_extension.applies": {
            "2029-01-01.2100-12-31": True
        }
    }, country_id="us")

# Policy 4
def get_ss_credit_reform(amount):
    return Reform.from_dict({
        "gov.contrib.crfb.ss_credit.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        "gov.contrib.crfb.ss_credit.amount.SINGLE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.JOINT": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.SEPARATE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.contrib.crfb.ss_credit.amount.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": amount
        },  
        "gov.irs.deductions.senior_deduction.amount": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0.85
        },
        "gov.irs.social_security.taxability.threshold.base.main.SINGLE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.JOINT": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SEPARATE": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.HEAD_OF_HOUSEHOLD": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.threshold.base.main.SURVIVING_SPOUSE": {
            "2026-01-01.2100-12-31": 0
        },
    }, country_id="us")

# Policy 5
def get_roth_style_swap_reform():
    return Reform.from_dict({
        "gov.irs.social_security.taxability.rate.base": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.irs.social_security.taxability.rate.additional": {
            "2026-01-01.2100-12-31": 0
        },
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2025-01-01.2100-12-31": True
        }
        }, country_id="us")

# Policy 6: Roth Style Swap with Phase-In
def get_roth_style_swap_with_phase_in_reform():
    """
    Beginning in 2026, employer payroll contributions phased into taxable income
    by 1 percentage point per year until full 7.65% is taxable.
    
    Starting in 2029, Social Security benefits taxation phased down by 5 percentage
    points per year (2029: 45/80, 2030: 40/75, ..., 2038: 0/35, ..., 2045: 0/0)
    """
    reform_dict = {
        # Enable employer payroll tax inclusion
        "gov.contrib.crfb.tax_employer_payroll_tax.in_effect": {
            "2026-01-01.2100-12-31": True
        },
        # Phase in employer payroll tax (1% per year from 2026 to 2033)
        "gov.contrib.crfb.tax_employer_payroll_tax.percentage": {
            "2026-01-01.2026-12-31": 0.1307,  # 1/7.65
            "2027-01-01.2027-12-31": 0.2614,  # 2/7.65
            "2028-01-01.2028-12-31": 0.3922,  # 3/7.65
            "2029-01-01.2029-12-31": 0.5229,  # 4/7.65
            "2030-01-01.2030-12-31": 0.6536,  # 5/7.65
            "2031-01-01.2031-12-31": 0.7843,  # 6/7.65
            "2032-01-01.2032-12-31": 0.9150,  # 7/7.65
            "2033-01-01.2100-12-31": 1        # Full amount
        },
        # Phase down Social Security benefits taxation starting 2029
        # Base rate starts at 50%, decreases by 5% per year
        "gov.irs.social_security.taxability.rate.base": {
            "2029-01-01.2029-12-31": 0.45,  # 45%
            "2030-01-01.2030-12-31": 0.40,  # 40%
            "2031-01-01.2031-12-31": 0.35,  # 35%
            "2032-01-01.2032-12-31": 0.30,  # 30%
            "2033-01-01.2033-12-31": 0.25,  # 25%
            "2034-01-01.2034-12-31": 0.20,  # 20%
            "2035-01-01.2035-12-31": 0.15,  # 15%
            "2036-01-01.2036-12-31": 0.10,  # 10%
            "2037-01-01.2037-12-31": 0.05,  # 5%
            "2038-01-01.2100-12-31": 0      # 0%
        },
        # Additional rate starts at 85%, decreases by 5% per year until 35%
        "gov.irs.social_security.taxability.rate.additional": {
            "2029-01-01.2029-12-31": 0.80,  # 80%
            "2030-01-01.2030-12-31": 0.75,  # 75%
            "2031-01-01.2031-12-31": 0.70,  # 70%
            "2032-01-01.2032-12-31": 0.65,  # 65%
            "2033-01-01.2033-12-31": 0.60,  # 60%
            "2034-01-01.2034-12-31": 0.55,  # 55%
            "2035-01-01.2035-12-31": 0.50,  # 50%
            "2036-01-01.2036-12-31": 0.45,  # 45%
            "2037-01-01.2037-12-31": 0.40,  # 40%
            "2038-01-01.2038-12-31": 0.35,  # 35%
            "2039-01-01.2039-12-31": 0.30,  # 30%
            "2040-01-01.2040-12-31": 0.25,  # 25%
            "2041-01-01.2041-12-31": 0.20,  # 20%
            "2042-01-01.2042-12-31": 0.15,  # 15%
            "2043-01-01.2043-12-31": 0.10,  # 10%
            "2044-01-01.2044-12-31": 0.05,  # 5%
            "2045-01-01.2100-12-31": 0      # 0%
        }
    }
    
    return Reform.from_dict(reform_dict, country_id="us")

In [11]:
# Pre-compute baselines for all years to avoid redundant calculations
print("Pre-computing baselines for all years...")
baselines = {}
years = list(range(2026, 2036))

for year in years:
    print(f"  Computing baseline for {year}...")
    baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    baseline_income = baseline.calculate("household_net_income", period=year)
    baselines[year] = baseline_income
    
print("Baseline computation complete!\n")

Pre-computing baselines for all years...
  Computing baseline for 2026...
  Computing baseline for 2027...
  Computing baseline for 2028...
  Computing baseline for 2029...
  Computing baseline for 2030...
  Computing baseline for 2031...
  Computing baseline for 2032...
  Computing baseline for 2033...
  Computing baseline for 2034...
  Computing baseline for 2035...
Baseline computation complete!



## Pre-compute Baselines
Computing baselines once for each year to improve performance

In [12]:
def calculate_yearly_impact(reform, year, baseline_income):
    """
    Calculate the budgetary impact for a given reform and year.
    Uses pre-computed baseline to avoid redundant calculations.
    Returns impact in billions (negative = cost, positive = savings)
    """
    try:
        if reform is not None:
            reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
            reformed_income = reformed.calculate("household_net_income", period=year)
            difference_income = reformed_income - baseline_income
            impact_billions = difference_income.sum() / 1e9
        else:
            # Return placeholder value if reform not implemented
            impact_billions = 0.0
            
        return round(impact_billions, 1)
    except Exception as e:
        print(f"Error calculating impact for year {year}: {e}")
        return 0.0

def save_results_incrementally(results, filename='policy_impacts_checkpoint.csv'):
    """
    Save results incrementally to avoid data loss
    """
    df = pd.DataFrame(results)
    df.to_csv(filename, index=False)
    return df

## Process All Policy Reforms
Iterate through each policy and calculate 10-year impacts

In [13]:
# Define years to analyze
years = list(range(2026, 2036))

# Initialize results list
all_results = []

# Policy configurations
policies = [
    {
        'id': 1,
        'name': 'Repeal Social Security Benefits Tax',
        'reform_func': get_repeal_social_security_reform,
        'has_variants': False
    },
    {
        'id': 2,
        'name': 'Flat Social Security Tax',
        'reform_func': get_flat_ss_tax_reform,
        'has_variants': False
    },
    {
        'id': 3,
        'name': 'Flat Social Security Tax with Bonus Senior Deduction Extension',
        'reform_func': get_flat_ss_tax_with_senior_deduction_extension_reform,
        'has_variants': False
    },
    {
        'id': 4,
        'name': 'Social Security Credit',
        'reform_func': get_ss_credit_reform,
        'has_variants': True,
        'variants': [300, 600, 900, 1200, 1500]
    },
    {
        'id': 5,
        'name': 'Roth Style Swap',
        'reform_func': get_roth_style_swap_reform,
        'has_variants': False
    },
    {
        'id': 6,
        'name': 'Roth Style Swap with Phase-In',
        'reform_func': get_roth_style_swap_with_phase_in_reform,
        'has_variants': False
    }
]

In [ ]:
# Process each policy with incremental saving
import os

# Check if checkpoint file exists to resume
checkpoint_file = 'policy_impacts_checkpoint.csv'
if os.path.exists(checkpoint_file):
    print(f"Found existing checkpoint file: {checkpoint_file}")
    existing_df = pd.read_csv(checkpoint_file)
    all_results = existing_df.to_dict('records')
    print(f"Loaded {len(all_results)} existing results")
    
    # Determine which policies/years have been completed
    completed_keys = set()
    for result in all_results:
        key = (result['policy_id'], result.get('credit_value'), result['year'])
        completed_keys.add(key)
else:
    all_results = []
    completed_keys = set()

save_counter = 0  # Counter to track when to save

for policy in policies:
    print(f"\nProcessing Policy {policy['id']}: {policy['name']}")
    
    if policy['has_variants']:
        # Handle policies with multiple variants (e.g., Social Security Credit)
        for variant in policy['variants']:
            print(f"  Variant: ${variant}")
            
            # Check if we need to process this variant
            needs_processing = False
            for year in years:
                key = (policy['id'], variant, year)
                if key not in completed_keys:
                    needs_processing = True
                    break
            
            if not needs_processing:
                print(f"    Already completed, skipping...")
                continue
                
            reform = policy['reform_func'](variant)
            
            for year in years:
                key = (policy['id'], variant, year)
                if key in completed_keys:
                    print(f"    Year {year}: Already computed, skipping...")
                    continue
                    
                print(f"    Year {year}: Computing...")
                impact = calculate_yearly_impact(reform, year, baselines[year])
                
                all_results.append({
                    'policy_id': policy['id'],
                    'policy_name': policy['name'],
                    'credit_value': variant,
                    'year': year,
                    'impact_billions': impact
                })
                
                save_counter += 1
                # Save every 10 calculations
                if save_counter % 10 == 0:
                    save_results_incrementally(all_results, checkpoint_file)
                    print(f"      Checkpoint saved ({len(all_results)} total results)")
    else:
        # Handle standard policies
        # Check if we need to process this policy
        needs_processing = False
        for year in years:
            key = (policy['id'], None, year)
            if key not in completed_keys:
                needs_processing = True
                break
        
        if not needs_processing:
            print(f"  Already completed, skipping...")
            continue
            
        try:
            reform = policy['reform_func']()
        except:
            # Use None if reform function not implemented
            reform = None
            print(f"  Note: Using placeholder values (reform not yet implemented)")
        
        for year in years:
            key = (policy['id'], None, year)
            if key in completed_keys:
                print(f"  Year {year}: Already computed, skipping...")
                continue
                
            print(f"  Year {year}: Computing...")
            impact = calculate_yearly_impact(reform, year, baselines[year])
            
            all_results.append({
                'policy_id': policy['id'],
                'policy_name': policy['name'],
                'credit_value': None,
                'year': year,
                'impact_billions': impact
            })
            
            save_counter += 1
            # Save every 10 calculations
            if save_counter % 10 == 0:
                save_results_incrementally(all_results, checkpoint_file)
                print(f"    Checkpoint saved ({len(all_results)} total results)")

# Final save
save_results_incrementally(all_results, checkpoint_file)
print(f"\nProcessing complete! Total results: {len(all_results)}")
print(f"Checkpoint saved to: {checkpoint_file}")


Processing Policy 1: Repeal Social Security Benefits Tax
  Year 2026: Computing...
  Year 2027: Computing...
  Year 2028: Computing...
  Year 2029: Computing...
  Year 2030: Computing...
  Year 2031: Computing...
  Year 2032: Computing...
  Year 2033: Computing...
  Year 2034: Computing...
  Year 2035: Computing...
    Checkpoint saved (10 total results)

Processing Policy 2: Flat Social Security Tax
  Year 2026: Computing...
  Year 2027: Computing...
  Year 2028: Computing...
  Year 2029: Computing...
  Year 2030: Computing...
  Year 2031: Computing...
  Year 2032: Computing...
  Year 2033: Computing...
  Year 2034: Computing...
  Year 2035: Computing...
    Checkpoint saved (20 total results)

Processing Policy 3: Flat Social Security Tax with Bonus Senior Deduction Extension
  Year 2026: Computing...
  Year 2027: Computing...
  Year 2028: Computing...
  Year 2029: Computing...
  Year 2030: Computing...
  Year 2031: Computing...
  Year 2032: Computing...
  Year 2033: Computing...
  

In [ ]:
# Create DataFrame from results
df_results = pd.DataFrame(all_results)

# Display summary statistics
print("\n=== Summary Statistics ===")
print(f"Total records: {len(df_results)}")
print(f"\nPolicies processed:")
for policy_id, group in df_results.groupby('policy_id'):
    policy_name = group['policy_name'].iloc[0]
    if group['credit_value'].notna().any():
        variants = group['credit_value'].dropna().unique()
        print(f"  {policy_id}. {policy_name} - {len(variants)} variants")
    else:
        print(f"  {policy_id}. {policy_name}")

# Display first few rows
print("\n=== First 10 rows ===")
display(df_results.head(10))


=== Summary Statistics ===
Total records: 100

Policies processed:
  1. Repeal Social Security Benefits Tax
  2. Flat Social Security Tax
  3. Flat Social Security Tax with Bonus Senior Deduction Extension
  4. Social Security Credit - 5 variants
  5. Roth Style Swap
  6. Roth Style Swap with Phase-In

=== First 10 rows ===


,policy_id,policy_name,credit_value,year,impact_billions
0,1,Repeal Social Security Benefits Tax,NaN,2026,82.7
1,1,Repeal Social Security Benefits Tax,NaN,2027,88.1
2,1,Repeal Social Security Benefits Tax,NaN,2028,93.2
3,1,Repeal Social Security Benefits Tax,NaN,2029,112.6
4,1,Repeal Social Security Benefits Tax,NaN,2030,121.0
5,1,Repeal Social Security Benefits Tax,NaN,2031,127.1
6,1,Repeal Social Security Benefits Tax,NaN,2032,133.7
7,1,Repeal Social Security Benefits Tax,NaN,2033,140.6
8,1,Repeal Social Security Benefits Tax,NaN,2034,148.0
9,1,Repeal Social Security Benefits Tax,NaN,2035,155.8


In [ ]:
# Save final results to CSV files
output_file = 'policy_impacts.csv'
df_results.to_csv(output_file, index=False)
print(f"Final results saved to {output_file}")

# Also save to the React app's public folder
react_output = 'policy-impact-dashboard/public/policy_impacts.csv'
df_results.to_csv(react_output, index=False)
print(f"Results also saved to {react_output}")

# Clean up checkpoint file if everything succeeded
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    print(f"Checkpoint file removed: {checkpoint_file}")

Final results saved to policy_impacts.csv
Results also saved to policy-impact-dashboard/public/policy_impacts.csv
Checkpoint file removed: policy_impacts_checkpoint.csv
